In [ ]:
import git
import pandas as pd




pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv").to_csv("ime_series_covid19_confirmed_US.csv", index=False)
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")



repoLocal = git.Repo( 'C:/Users/limc_/Documents/GitHub/Datos' )
print(repoLocal.git.status())
repoLocal.git.add(".")
try:
    repoLocal.git.commit(m='Update automatico via python')
except:
    pass

origin = repoLocal.remote(name='origin')
origin.push()

On branch master
Your branch is up to date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Johns_Hopkins-covid19/

nothing added to commit but untracked files present (use "git add" to track)
